In [1]:
% pylab inline

Populating the interactive namespace from numpy and matplotlib


In [145]:
def find_error_lin(x, w, y):
    E = 0.0
    E=dot(y.T,y)-2*dot(w.T,dot(x.T,y))+dot(dot(w.T, x.T),dot(x,w))
    return E

In [146]:
def lin_reg(X_train, X_test, y_train, y_test):
    w = np.linalg.pinv(X_train.T.dot(X_train)).dot(X_train.T).dot(y_train)
        
    return find_error_lin(X_test, w, y_test)

In [141]:
import random

def log_reg(X_train, X_test, y_train, y_test):
    train_samples, d = X_train.shape
    test_samples = X_test.shape[0]

    train_ind = random.sample(range(0,train_samples),train_samples)
    test_ind = random.sample(range(0,test_samples),test_samples)

    epochs = 100
    learning_rate = 0.0001

    weights = np.zeros(d+1)

    E_in = []

    for epoch in range(epochs):
        E_in_total = 0
        for ind in train_ind:
            extended_X_train = np.append(X_train[ind],1)
            cross_entropy_error = np.log(1+np.exp(-y_train[ind]*2*np.dot(weights.T, extended_X_train)))
            weights = weights + learning_rate * cross_entropy_error # does not work with minus sign!

            E_in_total += cross_entropy_error
        E_in_average = E_in_total / train_samples
        E_in.append(E_in_average)
#         print("EPOCH %d/%d \t Train error: %.5f" % (epoch, epochs, E_in_average))

    E_out_total = 0
    for ind in test_ind:
        extended_X_test = np.append(X_test[ind],1)
        cross_entropy_error = np.log(1+np.exp(-y_test[ind]*2*np.dot(weights.T, extended_X_test)))

        E_out_total += cross_entropy_error
    E_out_average = E_out_total / test_samples

    return E_out_average

In [114]:
def split_folds(X, y, folds=10):
    fold_size = int(X.shape[0] / folds)
    folds_dict = {}
    X_list = []
    y_list = []
    for i in range(folds-1):
        X_fold = X[fold_size*i : fold_size*i+fold_size]
        y_fold = y[fold_size*i : fold_size*i+fold_size]
        X_list.append(X_fold)
        y_list.append(y_fold)
        
    # take rest of the data
    X_fold = X[fold_size*(folds-1):]
    y_fold = y[fold_size*(folds-1):]
    X_list.append(X_fold)
    y_list.append(y_fold)
    
    folds_dict['X'] = X_list
    folds_dict['y'] = y_list
    
    return folds_dict

In [115]:
from sklearn import datasets
digits = datasets.load_digits()

In [116]:
X = digits.data
y = digits.target
print(X.shape)
print(y.shape)

(1797, 64)
(1797,)


In [133]:
folds = split_folds(X, y)

In [147]:
X_folds = np.array(folds['X'])
y_folds = np.array(folds['y'])

E_total_linear = []
E_total_logistic = []

for fold_num in range(len(folds['X'])):
    X_test = X_folds[fold_num]
    y_test = y_folds[fold_num]

    X_temp = np.delete(X_folds, fold_num)
    y_temp = np.delete(y_folds, fold_num)

    X_train = np.concatenate([X_temp[0]])
    for i in range(1,len(X_temp)):
        X_train = np.concatenate([X_train, X_temp[i]])

    y_train = np.concatenate([y_temp[0]])
    for i in range(1,len(y_temp)):
        y_train = np.concatenate([y_train, y_temp[i]])
        
    E_out_linear = lin_reg(X_train, X_test, y_train, y_test)
    E_total_linear.append(E_out_linear)
    
    E_out_logistic = log_reg(X_train, X_test, y_train, y_test)
    E_total_logistic.append(E_out_logistic)

In [148]:
E_total_logistic

[0.073574281735413194,
 0.073574281735413194,
 0.065829620500106531,
 0.069701951117759856,
 0.065829620500106531,
 0.061957289882453213,
 0.073574281735413194,
 0.065829620500106531,
 0.073574281735413194,
 0.063352161664081016]

In [149]:
E_total_linear

[962.51381012269894,
 533.62572829037072,
 631.4869358527385,
 626.32877596862636,
 698.09266902713989,
 510.14708026975859,
 546.07601788596094,
 803.82434287506021,
 892.37686019699959,
 907.12118390852902]